# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import nltk
nltk.download('wordnet')
import numpy as np
import pandas as pd
from nltk import sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk import  word_tokenize
from sqlalchemy import create_engine
from nltk.corpus import stopwords
import re
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier

[nltk_data] Downloading package wordnet to /Users/tangqi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('InsertTableName',con=engine)
df.head()

,id,message,original,genre,"('related',)","('request',)","('offer',)","('aid_related',)","('medical_help',)","('medical_products',)",...,"('aid_centers',)","('other_infrastructure',)","('weather_related',)","('floods',)","('storm',)","('fire',)","('earthquake',)","('cold',)","('other_weather',)","('direct_report',)"
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
X=df['message']
Y=df.drop(['id','message','original','genre'],axis=1)

### 2. Write a tokenization function to process your text data

In [6]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
def tokenize(text):
    detected_urls = re.findall(url_regex,text)
    for url in detected_urls:
        text =text.replace(url,'urlplaceholder')
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [7]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [8]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y)
pipeline.fit(X_train,Y_train)

/Users/tangqi/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/tangqi/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/tangqi/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/tangqi/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/tangqi/anaconda3/lib/python3.6/si

/Users/tangqi/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip..._score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=None))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [9]:
def report(pipeline,X_test,Y_test):
    Y_pred=pipeline.predict(X_test)
    for i,n in enumerate(Y_test):
        Y_true=Y_test[n]
        print(n)
        print(classification_report(Y_true,Y_pred[:,i]))

In [10]:
report(pipeline,X_test,Y_test)

('related',)
              precision    recall  f1-score   support

           0       0.64      0.37      0.47      1523
           1       0.82      0.93      0.88      4980
           2       0.73      0.16      0.26        51

   micro avg       0.80      0.80      0.80      6554
   macro avg       0.73      0.49      0.53      6554
weighted avg       0.78      0.80      0.78      6554

('request',)
              precision    recall  f1-score   support

           0       0.88      0.98      0.93      5439
           1       0.80      0.37      0.51      1115

   micro avg       0.88      0.88      0.88      6554
   macro avg       0.84      0.68      0.72      6554
weighted avg       0.87      0.88      0.86      6554

('offer',)
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6526
           1       0.00      0.00      0.00        28

   micro avg       1.00      1.00      1.00      6554
   macro avg       0.50      0.50    

/Users/tangqi/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           0       0.96      0.99      0.98      5940
           1       0.90      0.65      0.75       614

   micro avg       0.96      0.96      0.96      6554
   macro avg       0.93      0.82      0.87      6554
weighted avg       0.96      0.96      0.96      6554

('cold',)
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6430
           1       0.67      0.10      0.17       124

   micro avg       0.98      0.98      0.98      6554
   macro avg       0.82      0.55      0.58      6554
weighted avg       0.98      0.98      0.98      6554

('other_weather',)
              precision    recall  f1-score   support

           0       0.95      1.00      0.97      6206
           1       0.45      0.04      0.07       348

   micro avg       0.95      0.95      0.95      6554
   macro avg       0.70      0.52      0.52      6554
weighted avg       0.92      0.95      0.92  

### 6. Improve your model
Use grid search to find better parameters. 

In [11]:
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'vect', 'tfidf', 'clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'clf__estimator__bootstrap', 'clf__estimator__class_weight', 'clf__estimator__criterion', 'clf__estimator__max_depth', 'clf__estimator__max_features', 'clf__estimator__max_leaf_nodes', 'clf__estimator__min_impurity_decrease', 'clf__estimator__min_impurity_split', 'clf__estimator__min_samples_leaf', 'clf__estimator__min_samples_split', 'clf__estimator__min_weight_fraction_leaf', 'clf__estimator__n_estimators', 'clf__estimator__n_jobs', 'clf__estimator__oob_score', 'clf__estimator__random_state', 'clf__estimator__verbose', 'clf__estimator__

In [16]:
parameters = {'tfidf__use_idf': (True,False),
              'clf__estimator__n_estimators': [50, 100],
              'clf__estimator__min_samples_leaf': [2, 5,10]}

cv = GridSearchCV(pipeline, param_grid=parameters,cv=3,verbose=3)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [17]:
cv.fit(X_train,Y_train)

Fitting 3 folds for each of 12 candidates, totalling 36 fits
[CV] clf__estimator__min_samples_leaf=2, clf__estimator__n_estimators=50, tfidf__use_idf=True 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  clf__estimator__min_samples_leaf=2, clf__estimator__n_estimators=50, tfidf__use_idf=True, score=0.20491303021055843, total= 1.2min
[CV] clf__estimator__min_samples_leaf=2, clf__estimator__n_estimators=50, tfidf__use_idf=True 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.6min remaining:    0.0s


[CV]  clf__estimator__min_samples_leaf=2, clf__estimator__n_estimators=50, tfidf__use_idf=True, score=0.19346963686298443, total= 1.1min
[CV] clf__estimator__min_samples_leaf=2, clf__estimator__n_estimators=50, tfidf__use_idf=True 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  3.1min remaining:    0.0s


[CV]  clf__estimator__min_samples_leaf=2, clf__estimator__n_estimators=50, tfidf__use_idf=True, score=0.2076594446139762, total= 1.0min
[CV] clf__estimator__min_samples_leaf=2, clf__estimator__n_estimators=50, tfidf__use_idf=False 
[CV]  clf__estimator__min_samples_leaf=2, clf__estimator__n_estimators=50, tfidf__use_idf=False, score=0.19819957277998168, total=  59.5s
[CV] clf__estimator__min_samples_leaf=2, clf__estimator__n_estimators=50, tfidf__use_idf=False 
[CV]  clf__estimator__min_samples_leaf=2, clf__estimator__n_estimators=50, tfidf__use_idf=False, score=0.18843454379005187, total= 1.0min
[CV] clf__estimator__min_samples_leaf=2, clf__estimator__n_estimators=50, tfidf__use_idf=False 
[CV]  clf__estimator__min_samples_leaf=2, clf__estimator__n_estimators=50, tfidf__use_idf=False, score=0.2090326518156851, total=  58.5s
[CV] clf__estimator__min_samples_leaf=2, clf__estimator__n_estimators=100, tfidf__use_idf=True 
[CV]  clf__estimator__min_samples_leaf=2, clf__estimator__n_estimat

[Parallel(n_jobs=1)]: Done  36 out of  36 | elapsed: 55.5min finished


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip..._score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=None))]),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'tfidf__use_idf': (True, False), 'clf__estimator__n_estimators': [50, 100], 'clf__estimator__min_samples_leaf': [2, 5, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=3)

In [18]:
report(cv,X_test,Y_test)

('related',)
              precision    recall  f1-score   support

           0       0.80      0.18      0.30      1523
           1       0.79      0.99      0.88      4980
           2       0.67      0.04      0.07        51

   micro avg       0.79      0.79      0.79      6554
   macro avg       0.75      0.40      0.42      6554
weighted avg       0.79      0.79      0.74      6554

('request',)
              precision    recall  f1-score   support

           0       0.87      1.00      0.93      5439
           1       0.94      0.27      0.42      1115

   micro avg       0.87      0.87      0.87      6554
   macro avg       0.90      0.63      0.68      6554
weighted avg       0.88      0.87      0.84      6554

('offer',)
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6526
           1       0.00      0.00      0.00        28

   micro avg       1.00      1.00      1.00      6554
   macro avg       0.50      0.50    

/Users/tangqi/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           0       0.96      0.99      0.98      5940
           1       0.90      0.62      0.73       614

   micro avg       0.96      0.96      0.96      6554
   macro avg       0.93      0.81      0.86      6554
weighted avg       0.96      0.96      0.95      6554

('cold',)
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6430
           1       1.00      0.06      0.11       124

   micro avg       0.98      0.98      0.98      6554
   macro avg       0.99      0.53      0.55      6554
weighted avg       0.98      0.98      0.97      6554

('other_weather',)
              precision    recall  f1-score   support

           0       0.95      1.00      0.97      6206
           1       0.62      0.01      0.03       348

   micro avg       0.95      0.95      0.95      6554
   macro avg       0.79      0.51      0.50      6554
weighted avg       0.93      0.95      0.92  

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [19]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf',KNeighborsClassifier())
    ])
X_train,X_test,Y_train,Y_test=train_test_split(X,Y)
pipeline.fit(X_train,Y_train)
report(pipeline,X_train,Y_train)

('related',)
              precision    recall  f1-score   support

           0       0.76      0.54      0.64      4616
           1       0.87      0.95      0.91     14911
           2       0.93      0.29      0.44       135

   micro avg       0.85      0.85      0.85     19662
   macro avg       0.85      0.59      0.66     19662
weighted avg       0.84      0.85      0.84     19662

('request',)
              precision    recall  f1-score   support

           0       0.94      0.95      0.95     16331
           1       0.76      0.69      0.72      3331

   micro avg       0.91      0.91      0.91     19662
   macro avg       0.85      0.82      0.83     19662
weighted avg       0.91      0.91      0.91     19662

('offer',)
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     19572
           1       1.00      0.01      0.02        90

   micro avg       1.00      1.00      1.00     19662
   macro avg       1.00      0.51    

              precision    recall  f1-score   support

           0       0.91      0.96      0.93     15858
           1       0.78      0.61      0.69      3804

   micro avg       0.89      0.89      0.89     19662
   macro avg       0.84      0.79      0.81     19662
weighted avg       0.89      0.89      0.89     19662



/Users/tangqi/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 9. Export your model as a pickle file

In [21]:
import pickle

In [22]:
pickle.dump(cv, open('model.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.